In [1]:
import os
import sys
import numpy as np
import pandas as pd
import psycopg2 

sys.path.append('../')

from sqlalchemy import create_engine
from db_utils import create_table, populate_table, insert_into_table

In [2]:
# loading up the postgres credentials in a separate file. we could use environment variables
# but doing it this way to mix it up!
user = os.environ['DB_USER']
password = os.environ['DB_PASSWORD']
host = os.environ['DB_HOST']
database = os.environ['DB_NAME']
port = os.environ['DB_PORT']
    

URI = f'postgresql://{user}:{password}@{host}:{port}/{database}'

In [ ]:
# getting the directory path
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
nfl_pbp_dir = 'data/pbp'
nfl_depth_chart_dir = 'data/depth_charts'

# loading all of the parquets files at once using the directory path
# replacing backslashes with empty strings to avoid csv errors
df = pd.read_parquet(f'{parent_dir}/{nfl_pbp_dir}')
df['desc'] = df['desc'].str.replace('\\', '', regex=True)
df['season'] = [int(x.split('_')[0]) for x in df.game_id]

print("the shape of the pbp df is:", df.shape)
print("the memory usage of the pbp df is :", df.memory_usage(deep=True).sum() / 1024**2, "MB")

# dropping columns to reduce the memory required in the hopes of populating the db
# all at once, but it didn't work. I'll have to do it in chunks.
unneeded_cols = ['nfl_api_id', 'old_game_id', 'home_coach', 'away_coach', 'game_stadium', 'weather']
df.drop(columns=[x for x in df.columns if 'lateral' in x], inplace=True)
df.drop(columns=[x for x in df.columns if 'player_2' in x], inplace=True)
df.drop(columns=[x for x in df.columns if 'total' in x], inplace=True)
df.drop(columns=unneeded_cols, inplace=True)

print("the memory usage of the pbp after dropping is :", df.memory_usage(deep=True).sum() / 1024**2, "MB")

In [ ]:
# creating the pbp table
create_table(df=df, table_name='pbp', URI=URI)

# creating a subset of the pbp df to test the populate_table function
testdf = df.iloc[:10000]
populate_table(df=testdf, table_name='pbp', URI=URI)

In [ ]:
# using the insert function to get the pbp  into the database
# quickly - estimated time with 50k chunk sizes is 2.5 minutes
# the print statements keep track of the progress and 
# enable error identification

chunks = [x for x in range(10000, len(df), 50000)]

for i in range(len(chunks)):
    
    if i < (len(chunks) - 1):
        print(i)
        df_ = df.iloc[chunks[i] : chunks[i+1]].copy()
        print("chunk final index:", df_.index[-1])
        try:
            insert_into_table(df=df_, table_name='pbp', URI=URI)
        except:
            print("error in insert", i)
            print(df_.index[-1])
            continue
    else:
        print(i)
        df_ = df.iloc[chunks[i]:].copy()
        print("last chunk final index:", df_.index[-1])
        try:
            insert_into_table(df=df_, table_name='pbp', URI=URI)
        except:
            print("error for some other reason", i)

In [ ]:
# loading and inserting the depth chart data into the database
# depth chart df needed to have line breaks replaced

depth_charts = pd.read_parquet(f'{parent_dir}/{nfl_depth_chart_dir}')
depth_charts['depth_position'] = depth_charts['depth_position'].str.replace("\n", '')

create_table(df=depth_charts, table_name='depth_charts', URI=URI)
populate_table(df=depth_charts, table_name='depth_charts', URI=URI)

In [ ]:
# loading player stats data into the database
player_stats = pd.read_parquet(f'{parent_dir}/data/player_stats')

create_table(df=player_stats, table_name='player_stats', URI=URI)
populate_table(df=player_stats, table_name='player_stats', URI=URI)

In [ ]:
# loading nextgen data into the database
nextgen = pd.read_parquet(f'{parent_dir}/data/nextgen')

create_table(df=nextgen, table_name='nextgen', URI=URI)
populate_table(df=nextgen, table_name='nextgen', URI=URI)

In [ ]:
# loading misc data into the database
misc = pd.read_parquet(f'{parent_dir}/data/misc')

create_table(df=misc, table_name='misc', URI=URI)
populate_table(df=misc, table_name='misc', URI=URI)

In [ ]:
# loading snap counts data into the database
snaps = pd.read_parquet(f'{parent_dir}/data/snap_counts')

create_table(df=snaps, table_name='snaps', URI=URI)
populate_table(df=snaps, table_name='snaps', URI=URI)

In [ ]:
# loading Lee Sharpe's data into the database
lee_sharpe = 'https://raw.githubusercontent.com/nflverse/nfldata/master/data/games.csv'

ls_cols = ['game_id', 'overtime', 'home_rest', 'away_rest', 'div_game','roof', 
           'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'referee']

ls = pd.read_csv(lee_sharpe, usecols=ls_cols)
ls = ls.sort_values(by='game_id')

create_table(df=ls, table_name='lee_sharpe', URI=URI)
populate_table(df=ls, table_name='lee_sharpe', URI=URI)